In [ ]:
%cd /home/parthgandhi/TradeBot

In [ ]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange

In [ ]:
end_date = "2025-12-26"

In [ ]:
classify_df = pl.scan_csv("/home/parthgandhi/Downloads/charts_maze_scanner.csv").rename(
    {
        "Stock Name": "symbol",
        "RS Rating": "rs_rating",
        "Basic Industry": "basic_industry",
        "Market Cap(Cr.)": "market_cap_cr",
        "1 Month Returns(%)": "1_mo_rtr_pct",
        "3 Month Returns(%)": "3_mo_rtr_pct",
        "% from 52W High": "pct_from_52w_high",
    }
)

In [ ]:
filters_path = StorageLayout.filters_dir(
    run_date=end_date, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

basic_filter = (
    pl.scan_csv(filters_path / "basic_filter.csv")
    .with_columns(
        pl.col("timestamp")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.f")
        .cast(pl.Date)
        .alias("timestamp")
    )
    .select("timestamp", "symbol")
)

In [ ]:
df_list = []
for filter_type in ["sma_200", "adr", "pullback", "reversal", "vcp"]:
    df = (
        pl.scan_csv(filters_path / f"{filter_type}_filter.csv")
        .with_columns(pl.lit(True).alias(f"{filter_type}_filter_flag"))
        .select("symbol", f"{filter_type}_filter_flag")
    )
    df_list.append(df)

In [ ]:
res = (
    basic_filter.join(df_list[0], on="symbol", how="left")
    .join(df_list[1], on="symbol", how="left")
    .join(df_list[2], on="symbol", how="left")
    .join(df_list[3], on="symbol", how="left")
    .join(df_list[4], on="symbol", how="left")
    .with_columns(cs.ends_with("flag").fill_null(False))
    .join(classify_df, on="symbol", how="left")
    .collect()
)

In [ ]:
print(f"Before RS filter: {res.shape}")
res.write_csv(filters_path / "overall_filter_result.csv")
res = res.filter(pl.col("rs_rating") >= 70)
print(f"After RS filter: {res.shape}")

In [ ]:
industry_analysis = (
    res.lazy()
    .group_by("basic_industry")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr").sum()
            )
            .round(2)
            .alias(f"rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .sort("1_mo_avg_weight_rtr_pct", descending=True)
    .with_columns(
        pl.mean_horizontal(cs.exclude("basic_industry", "symbols_count"))
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    )
    .collect()
)

In [ ]:
industry_analysis.write_csv(filters_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [ ]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

In [ ]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1).to_pandas()